# Predicting Heart failure

## The dataset

The dataset used in this projects was published to Kaggle by ... in 2020? and contains hospital data from xxx patients admitted to the xxx Hospital in Pakistan between XXX and XXX with cardiac symptoms.

In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Import data as .csv file

In [ ]:
#Dataset imported from: https://www.kaggle.com/datasets/asgharalikhan/mortality-rate-heart-patient-pakistan-hospital

HF_Dataset = 'HFDataset.csv'
HFdf_full = pd.read_csv(HF_Dataset)

For the purpose of this project, I have used a dataset downloaded from Kaggle as a .csv file to the Jupyter directory and uploaded to the local environement. Had I used a dataset from a different source, I could have used the source API to pull the dataset directly. As this is not the case here, I have included an unrelated example to showcase how I would go about accessing data using an API. I have pulled an interesting dataset using the Smart Dublin API: Fire and Ambulance Annual Incident Activity Log published by data.gov.ie


Initially used urllib library to download the file via the Smart Dublin API, however I discovered that this throws an error as Anaconda has replaced 'urllib' with 'requests'. I have imported the 'requests' library instead.

In [ ]:
import requests
import pandas as pd

#define dataset url and 
url = "https://data.smartdublin.ie/api/3/action/datastore_search"
resource_id = "b9281d81-ccde-4c1e-b8db-b2b9f7e6eea8"
limit = 5

#Send get request, storing downloaded dataset as variable 'response'
response = requests.get(url, params={"resource_id": resource_id, "limit": limit})

#Extract 'results' and 'records' from the received .json file
data = response.json()
records = data["result"]["records"]

#Convert the resulting list into DataFrame for further analysis.
df2= pd.DataFrame(records)
print(df2.head())

## Preliminary data analysis

In [ ]:
#Use .head() method on our dataframe to take an exploratory look at what data in holds. 
print(HFdf_full.head())

In [ ]:
#Use .info() method to get information on the type of data in the dataframe
print(HFdf_full.info())

## Clean the Data

Not all columns are intuitive as to what the contained data represents. Without additional information on the dataset these columns do not provide any useful information and are dropped from further use.

In [ ]:
# For the purposes of this analysis, "Thalach", "exang", "oldpeak", "slope", "ca", "thal", "num", "SK", /
# "SK.React", and "Reaction" columns will be dropped.

drop columns
#save resulting dataframe as HFdf

In [23]:
#Check for missing values
HFdf.isna()

NameError: name 'HFdf' is not defined

In [ ]:
#Count number of missing data points
HFdf.isna().sum

In [22]:
#Missing data points can be visualized in a matrix using the missingno package
import missingno as msno
import matplotlib.pyplot as plt
msno.matrix(HFdf)
plt.show

NameError: name 'HFdf' is not defined

In [ ]:
#Change datatypes from yes/no object to booolean

In [ ]:

#Check for duplicates

#use WBC and BGR to filter for duplicate plts

The dataset is parsed into groups according to relevance and saved. This will promote ease of use down the line.

In [ ]:
#Extracted column titles as a list
titles = HFdf.columns.tolist()

In [ ]:
#For ease-of use, printed the index value for each column title
for index, item in enumerate(titles):
    print(f"{index}: {item}")

In [ ]:
#Dataframe containing demographics data

df_demographics = HFdf[titles[:3]]

In [ ]:
#Dataframe containing relevant past medical history data

df_pmhx = HFdf[titles[8:18]]

In [ ]:
#Dataframe containing laboratory results for blood work other than standard CBC

df_bloods_chem = HFdf[titles[18:27]]

In [ ]:
#Dataframe containing laboratory results for standard CBC blood work

df_bloods_cbc = HFdf[titles[27:39]]

In [ ]:
#Dataframe containing admissions symptom data

df_sxoa = HFdf[titles[39:49]]

In [24]:
#Dataframe containing outcome data

df_sxoa = HFdf[titles[-2:]]

NameError: name 'HFdf' is not defined

## Interpretation of columns with unique string data

Use ReGex to match string entries in columns containing data on symptoms present on admission ("CO") and established diagnosis ("Diagnosis")

In [18]:
#Can I make this a for loop?


#Use df[].unique() to determine all unique answers to “other”, “CO”, “Diagnosis”,
Unique_other = HFdata_df["other"].unique()
Unique_CO = HFdata_df["CO"].unique()
Unique_Diagnosis = HFdata_df["Diagnosis"].unique()


NameError: name 'HFdata_df' is not defined

In [19]:
import numpy as np
Col_unique_list = ["Others", "CO", "Diagnosis",]

for name in Col_unique_list:
    unique_content = HFdata_df[name].unique()
    print(unique_content)

NameError: name 'HFdata_df' is not defined

In [ ]:
#Use ReGex to filter for relevant keywords
Relevant_keywords_CO = pd.list[chest pain, SOB,]
Relevant_keywords_Diagnosis = 
Relevant_keywords_other = 

Use ReGex to search column and match symptoms which were present on admission.

In [3]:
#Use ReGex to match specific symptoms - Chest pain 
import pandas as pd
import re

#Convert data type from object to string
As_ser = HFdata_df["CO"]
As_str = As_ser.astype(str).str
print(type(As_str))

#Make string lowercase
As_lowerc = As_str.lower()

# Create a new column called "CO_chest_pain" to store boolean value, True if symptom present, otherwise False
HFdata_df['CO_chest_pain'] = As_lowerc.str.contains(r'\w*\s?chest\s?pain\w*', regex=True)

print(HFdata_df['CO_chest_pain'].head())

NameError: name 'HFdata_df' is not defined

In [ ]:
#Use ReGex to match specific symptoms - Sweating

In [ ]:
#Use ReGex to match specific symptoms - Nausea or Vomiting

In [5]:
#Pull only rows which did not contain either of the three presenting symptomes found by ReGex 
# Save these rows of unique atypical presentations as HFdata_df["CO_unique_atypical"]

In [ ]:
#Histogram showing percentage of pts exhibiting most common symptoms - by age group, gender.

#Maybe put this lower in with Basic stat analysis...

Use ReGex to search column containing free-form string data on established diagnosis ("Diagnosis")

In [4]:
#Use ReGex to match specific Diagnosis - A/W, .... maybe too much, not enough time ....

## Basic Statistical Analysis 

In [6]:
#Histogram showing breakdown of dataset by: Gender, Age/Age Group

In [7]:
#MOrtality by Age.Group and location

In [8]:
#Mortality hist? correlation? by comorbidity : Depression, Hyperlipidemia, Smoking, DM, HTN

In [9]:
#Correlation: BP, BGR - scatter

In [10]:
#Correlation: Bloods Chem?

In [11]:
#Correlation:Diagnosis? - if regex done

In [12]:
#Correlation: CP value ******** linear regression

In [13]:
#Correlation: Followup? - maybe

## Machine Learning

Using a supervised learning algorythm, 

In [17]:
#...

Hyperparameter tuning

In [16]:
#...